
# Model Registry (Registro de Modelos) con MLflow

El Registro de Modelos de MLflow (MLflow Model Registry) es un centro colaborativo donde los equipos pueden compartir modelos de Machine Learning, trabajar juntos desde la experimentación hasta las pruebas y la producción, integrarse con flujos de trabajo de aprobación y gobernanza, y monitorear los despliegues de ML y su rendimiento.

Esta lección explora cómo gestionar modelos utilizando el registro de modelos de MLflow en un entorno open-source.

## En esta lección:
  - Registrarás un modelo usando MLflow.
  - Desplegarás ese modelo a un entorno simulado de "Producción".
  - Actualizarás un modelo en "Producción" a una nueva versión, incluyendo una fase de "Staging" (pruebas).
  - Archivarás y eliminarás modelos.

### El Registro de Modelos

El componente de Registro de Modelos de MLflow es un almacén centralizado de modelos, un conjunto de APIs y una interfaz de usuario para gestionar de forma colaborativa el ciclo de vida completo de un modelo de MLflow. Proporciona linaje del modelo (qué experimento y ejecución de MLflow produjo el modelo), versionado, transiciones de fase (por ejemplo, de `Staging` a `Production`), anotaciones (como comentarios y etiquetas) y gestión de despliegues.

El registro de modelos tiene las siguientes características:
* **Repositorio Central:** Registra modelos de MLflow. Un modelo registrado tiene un nombre único, versión, fase y otros metadatos.
* **Versionado de Modelos:** Rastrea automáticamente las versiones de los modelos registrados cuando se actualizan.
* **Fase del Modelo:** Asigna fases predefinidas o personalizadas a cada versión del modelo, como `Staging` y `Production`, para representar el ciclo de vida del modelo.
* **Transiciones de Fase del Modelo:** Registra nuevos eventos o cambios como actividades que registran automáticamente usuarios, cambios y metadatos adicionales como comentarios.
* **Integración con Flujos de CI/CD:** Utiliza las transiciones de fase para solicitar, revisar y aprobar cambios como parte de los pipelines de CI/CD para un mejor control y gobernanza.



### Configuración Inicial: Conexión con el Servidor MLflow
Antes de empezar, necesitamos indicarle a nuestro script dónde se encuentra el servidor de MLflow que iniciaste previamente.


In [3]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from mlflow.models.signature import infer_signature
from mlflow.tracking.client import MlflowClient
import time

In [4]:
# Establece la URI del servidor de tracking de MLflow
# Asegúrate de que coincida con la URL donde iniciaste 'mlflow ui'
mlflow.set_tracking_uri("http://127.0.0.1:5000")

print(f"MLflow Version: {mlflow.__version__}")

MLflow Version: 3.2.0


### Registrar un Modelo

El siguiente flujo de trabajo se puede realizar tanto desde la interfaz de usuario como puramente con Python. Este notebook usará únicamente Python.

Primero, entrenaremos un modelo y lo registraremos en MLflow.


In [5]:
# Cargar datos directamente desde su fuente original (UCI Machine Learning Repository)
white_wine = pd.read_csv("winequality-white.csv", sep=";")
red_wine = pd.read_csv("winequality-red.csv", sep=";")

# Añadir una columna para identificar el tipo de vino
red_wine['is_red'] = 1
white_wine['is_red'] = 0

data = pd.concat([red_wine, white_wine], axis=0)

In [6]:
# Reemplazar espacios en los nombres de las columnas por guiones bajos
data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

# Convertir el problema a clasificación binaria (calidad alta >= 7)
high_quality = (data.quality >= 7).astype(int)
data['quality'] = high_quality

In [7]:
# Separar datos de entrenamiento y prueba
train, test = train_test_split(data, random_state=123)
X_train = train.drop(["quality"], axis=1)
X_test = test.drop(["quality"], axis=1)
y_train = train.quality
y_test = test.quality

# Parámetros del modelo
n_estimators = 100
max_depth = 5

# Entrenar el modelo
rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
rf.fit(X_train, y_train)

# Inferir la firma del modelo y crear un ejemplo de entrada
input_example = X_train.head(3)
signature = infer_signature(X_train, pd.DataFrame(y_train))

c:\Users\alopezmoreno\OneDrive - Microsoft\Desktop\MLOps-UTP\mlops-course\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [8]:
# Iniciar una ejecución de MLflow y registrar el modelo
with mlflow.start_run(run_name="Modelo RF - v1") as run:
    mlflow.sklearn.log_model(rf, "modelo-rf", input_example=input_example, signature=signature)
    mlflow.log_metric("auc", roc_auc_score(y_test, rf.predict(X_test)))
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    run_id = run.info.run_id
    print(f"Modelo guardado en la ejecución: {run_id}")

2025/08/14 12:35:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/14 12:35:38 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Modelo guardado en la ejecución: 90562d3047394b4ebbb65fe0aa2845ab
🏃 View run Modelo RF - v1 at: http://127.0.0.1:5000/#/experiments/112621643149097054/runs/90562d3047394b4ebbb65fe0aa2845ab
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/112621643149097054


Creamos un nombre de modelo único para no colisionar con otros modelos en el registro.

In [9]:
# Usamos un sufijo para asegurar que el nombre del modelo es único
suffix = "aml"
model_name = f"clasificador-vinos-rf_{suffix}"
print(f"Nombre del modelo: {model_name}")

Nombre del modelo: clasificador-vinos-rf_aml


In [10]:
# Ahora, registramos el modelo que acabamos de entrenar en el Registro de Modelos.
model_uri = f"runs:/{run_id}/modelo-rf"

# Registramos el modelo y obtenemos sus detalles
model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

print(f"Modelo '{model_details.name}' registrado. Versión: {model_details.version}")

Successfully registered model 'clasificador-vinos-rf_aml'.
2025/08/14 12:36:34 WARNING mlflow.tracking._model_registry.fluent: Run with id 90562d3047394b4ebbb65fe0aa2845ab has no artifacts at artifact path 'modelo-rf', registering model based on models:/m-740ac306c8954518b85002975e974548 instead
2025/08/14 12:36:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: clasificador-vinos-rf_aml, version 1


Modelo 'clasificador-vinos-rf_aml' registrado. Versión: 1


Created version '1' of model 'clasificador-vinos-rf_aml'.


**Ahora puedes ir a tu navegador en `http://127.0.0.1:5000` y hacer clic en la pestaña "Models" para explorar el modelo registrado.**

Verás lo siguiente:

* Quién entrenó el modelo y el código fuente asociado.
* Un historial de acciones realizadas sobre este modelo.
* El modelo registrado como la "Version 1".

### Actualizar Metadatos del Modelo

Podemos añadir descripciones al modelo y a sus versiones para documentar su propósito.


In [11]:
client = MlflowClient()

# Esperamos un poco para asegurarnos de que el registro se ha completado
print("Esperando 10 segundos para que el registro se complete...")
time.sleep(10)

# Verificamos el estado del modelo
model_version_details = client.get_model_version(name=model_name, version=1)
print(f"Estado inicial del modelo: {model_version_details.status}")

Esperando 10 segundos para que el registro se complete...
Estado inicial del modelo: READY


In [ ]:
# Añadimos una descripción general al modelo registrado
client.update_registered_model(
    name=model_details.name,
    description="Este modelo clasifica la calidad del vino (alta/baja) basándose en datos fisicoquímicos."
)

# Añadimos una descripción específica a esta versión
client.update_model_version(
    name=model_details.name,
    version=model_details.version,
    description="Versión inicial construida con un RandomForest de 100 árboles y profundidad 5."
)
print("Descripciones del modelo y de la versión actualizadas.")

### Desplegar un Modelo

El Registro de Modelos de MLflow define varias fases (`stage`): `None`, `Staging`, `Production` y `Archived`. Cada fase tiene un significado. Por ejemplo, `Staging` es para pruebas, mientras que `Production` es para modelos que han completado las pruebas y han sido desplegados.

Ahora, haremos la transición del modelo a la fase de **`Production`**.

In [ ]:
# Usando el nuevo enfoque con alias en lugar de stages
client.set_registered_model_alias(
    name=model_details.name,
    alias="production",
    version=model_details.version
)
print(f"Modelo versión {model_details.version} asignado al alias 'production'.")

C:\Users\alopezmoreno\AppData\Local\Temp\ipykernel_18992\1217307799.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


KeyboardInterrupt: 

In [ ]:
# %% [markdown]


# %%


# %% [markdown]
# Podemos verificar los alias actuales del modelo para confirmar el cambio.

# %%
# Obtener información del modelo registrado
registered_model = client.get_registered_model(name=model_details.name)
print(f"Alias del modelo '{model_details.name}':")
for alias in registered_model.aliases:
    print(f"  - {alias}: versión {registered_model.aliases[alias]}")

# %% [markdown]
# ### Cargar y Usar un Modelo desde el Registro
#
# Ahora podemos cargar el modelo directamente desde el registro usando su alias. Cargarlo como `pyfunc` nos permite usarlo independientemente del framework con el que fue entrenado (en este caso, scikit-learn).

# %%
# La URI para cargar un modelo con un alias específico es "models:/<nombre_modelo>@<alias>"
model_production_uri = f"models:/{model_name}@production"

print(f"Cargando modelo en producción desde la URI: '{model_production_uri}'")
production_model = mlflow.pyfunc.load_model(model_production_uri)

# Aplicamos el modelo para hacer predicciones
predicciones = production_model.predict(X_test)
print("Predicciones realizadas con el modelo de producción:")
print(predicciones)

# %% [markdown]
# ### Desplegar una Nueva Versión del Modelo
#
# El ciclo de vida de un modelo no termina con el primer despliegue. A menudo, reentrenaremos el modelo con nuevos datos, algoritmos o hiperparámetros. El Registro de Modelos nos permite gestionar este proceso de forma ordenada.
#
# Ahora entrenaremos un modelo más potente y lo registraremos como una nueva versión.

# %%
# Nuevos hiperparámetros
n_estimators = 300
max_depth = 10

rf_v2 = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
rf_v2.fit(X_train, y_train)

input_example = X_train.head(3)
signature = infer_signature(X_train, pd.DataFrame(y_train))

with mlflow.start_run(run_name="Modelo RF - v2") as run:
    # Al usar 'registered_model_name', MLflow registra el modelo
    # y crea automáticamente una nueva versión.
    mlflow.sklearn.log_model(
        sk_model=rf_v2,
        artifact_path="sklearn-model-v2",
        registered_model_name=model_name, # <-- Esto registra la nueva versión
        input_example=input_example,
        signature=signature
    )
    mlflow.log_metric("auc", roc_auc_score(y_test, rf_v2.predict(X_test)))
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    
    run_id_v2 = run.info.run_id
    print(f"Nueva versión del modelo guardada en la ejecución: {run_id_v2}")

# %% [markdown]
# **Vuelve a revisar la interfaz de MLflow.** Verás que ha aparecido la "Version 2" del modelo.
#
# Ahora, vamos a gestionar su ciclo de vida.

# %%
# Buscamos la última versión del modelo
model_version_infos = client.search_model_versions(f"name = '{model_name}'")
new_model_version = max([model_version_info.version for model_version_info in model_version_infos])
print(f"La nueva versión del modelo es: {new_model_version}")

# Esperamos por si el registro aún está pendiente
print("Esperando 10 segundos...")
time.sleep(10)

# Añadimos una descripción a la nueva versión
client.update_model_version(
    name=model_name,
    version=new_model_version,
    description="Este modelo es un clasificador aleatorio con 300 árboles de decisión y una profundidad máxima de 10."
)

# Asignamos un alias 'staging' a la nueva versión para pruebas
client.set_registered_model_alias(
    name=model_name,
    alias="staging",
    version=new_model_version
)
print(f"Versión {new_model_version} asignada al alias 'staging'.")

# %% [markdown]
# Una vez que el modelo en `staging` ha pasado todas las pruebas de un pipeline de CI/CD, podemos promoverlo a `production`. Al hacerlo, el alias `production` se moverá automáticamente a la nueva versión.

# %%
# Promovemos la nueva versión a production
client.set_registered_model_alias(
    name=model_name,
    alias="production",
    version=new_model_version
)

# Opcionalmente, podemos eliminar el alias staging
client.delete_registered_model_alias(
    name=model_name,
    alias="staging"
)

print(f"Versión {new_model_version} promovida a 'production'. El alias 'staging' ha sido eliminado.")


# %% [markdown]
# ### Gestionar Modelos con Alias
#
# Con el nuevo sistema de alias, la gestión de modelos es más flexible. Podemos tener múltiples alias apuntando a diferentes versiones según nuestras necesidades.

# %% [markdown]
# Ahora podemos gestionar las versiones de modelos de manera más flexible. En lugar de eliminar versiones, simplemente gestionamos los alias.

# %%
# Verificar todos los alias actuales
registered_model = client.get_registered_model(name=model_name)
print(f"Alias actuales del modelo '{model_name}':")
for alias_name, version in registered_model.aliases.items():
    print(f"  - {alias_name}: versión {version}")

# Si queremos "archivar" una versión, simplemente removemos todos sus alias
# y opcionalmente podemos agregar un alias 'archived' si lo deseamos
if len(registered_model.aliases) > 0:
    # Crear un alias 'archived' para la versión anterior (si existe)
    try:
        client.set_registered_model_alias(
            name=model_name,
            alias="archived",
            version="1"  # Version anterior
        )
        print("Versión 1 marcada como 'archived'.")
    except Exception as e:
        print(f"No se pudo archivar la versión 1: {e}")

# %% [markdown]
# Por último, si el modelo ya no es relevante, podemos eliminar todo el registro del modelo. Para ello, primero debemos eliminar todos los alias.

# %%
# Eliminar todos los alias antes de eliminar el modelo
registered_model = client.get_registered_model(name=model_name)
for alias_name in registered_model.aliases.keys():
    client.delete_registered_model_alias(
        name=model_name,
        alias=alias_name
    )
    print(f"Alias '{alias_name}' eliminado.")

# Ahora eliminamos todo el modelo registrado
client.delete_registered_model(model_name)
print(f"El modelo registrado '{model_name}' ha sido eliminado.")